In [13]:
import sys

sys.path.append("/home/emm06/birdclef-2023/vendor/BirdNET-Analyzer/")
import os
import argparse
import datetime
import numpy as np

# import importlib
import traceback

from multiprocessing import Pool
import importlib

config = importlib.import_module("vendor.BirdNET-Analyzer.config")
analyze = importlib.import_module("vendor.BirdNET-Analyzer.analyze")
model = importlib.import_module("vendor.BirdNET-Analyzer.model")

# from vendor.BirdNET_Analyzer import analyze
import keras

##import model
import os

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.utils.vis_utils import plot_model
import numpy as np

import warnings

# warnings.filterwarnings("ignore")
# import tflite2onnx
import tf2onnx

# import config as cfg
import tensorflow as tf

print(tf.__version__)
print(tf.__file__)

print(tf2onnx.__file__)

2.11.0
/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/tensorflow/__init__.py
/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/tf2onnx/__init__.py


In [11]:
# Version 1.15 Tensorflow - issues with readiing the protobuff file
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    # Then, we import the graph_def into a new Graph and returns it
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


test = load_graph(
    "/home/emm06/birdclef-2023/vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb"
)

DecodeError: Error parsing message with type 'tensorflow.GraphDef'

In [3]:
! mmconvert -sf tensorflow -iw  ../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb --inNodeName INPUT --inputShape 144000 --dstNodeName CLASS_DENSE_1 -df pytorch -om  tf2torch_saved_model

Traceback (most recent call last):
  File "/home/emm06/birdclef-2023/venv/bin/mmconvert", line 8, in <module>
    sys.exit(_main())
  File "/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/mmdnn/conversion/_script/convert.py", line 102, in _main
    ret = convertToIR._convert(ir_args)
  File "/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/mmdnn/conversion/_script/convertToIR.py", line 59, in _convert
    parser = TensorflowParser2(args.weights, inputshape, args.inNodeName, args.dstNodeName)
  File "/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/mmdnn/conversion/tensorflow/tensorflow_frozenparser.py", line 120, in __init__
    tensorflow.reset_default_graph()
AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'


Next part is with Tensorflow 2x, everything works fine...but when running the following command in bash, there are compatibility issues with mmdnn converter since it works in Tensorflow 1.15:

!mmconvert -sf tensorflow -iw  vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/saved_model.pb --inNodeName INPUT --inputShape 144000 --dstNodeName CLASS_DENSE_1 -df pytorch -om  tf2torch_saved_model

In [4]:
export_path = "/home/emm06/birdclef-2023/vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"

In [5]:
pb_model = keras.models.load_model(export_path)

In [6]:
pb_model.model.layers[0].get_config()

{'batch_input_shape': (None, 144000),
 'dtype': 'float32',
 'sparse': False,
 'ragged': False,
 'name': 'INPUT'}

In [7]:
pb_model.model.layers[0].get_config()["batch_input_shape"]

(None, 144000)

In [8]:
pb_model.model.layers[-1].get_config()

{'name': 'CLASS_DENSE_1',
 'trainable': True,
 'dtype': 'float32',
 'units': 3337,
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'HeNormal', 'config': {'seed': 42}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [9]:
pb_model.model.layers

In [10]:
[layer.name for layer in pb_model.model.layers]

['INPUT',
 'ADVANCED_SPEC1',
 'BNORM_SPEC_NOQUANT',
 'CONV_0',
 'BNORM_0',
 'ACT_0',
 'pool_0_MAX',
 'pool_0_AVG',
 'pool_0_CONCAT',
 'pool_0_ACT_QUANT',
 'pool_0_CONV',
 'BLOCK_1-1_CONV_1',
 'BLOCK_1-1_BN_1',
 'BLOCK_1-1_ACT_1',
 'BLOCK_1-1_ACT_QUANT',
 'BLOCK_1-1_CONV_3',
 'BLOCK_1-1_BN_3',
 'BLOCK_1-2_CONV_1',
 'BLOCK_1-2_BN_1',
 'BLOCK_1-2_ACT_1',
 'BLOCK_1-2_ACT_QUANT',
 'BLOCK_1-2_CONV_3',
 'BLOCK_1-2_BN_3',
 'BLOCK_1-2_ADD',
 'BLOCK_1-3_CONV_1',
 'BLOCK_1-3_BN_1',
 'BLOCK_1-3_ACT_1',
 'BLOCK_1-3_ACT_QUANT',
 'BLOCK_1-3_CONV_3',
 'BLOCK_1-3_BN_3',
 'BLOCK_1-3_ADD',
 'BLOCK_2-1_CONV_1',
 'BLOCK_2-1_BN_1',
 'BLOCK_2-1_ACT_1',
 'BLOCK_2-1_ACT_QUANT',
 'BLOCK_2-1_CONV_3',
 'BLOCK_2-1_BN_3',
 'BLOCK_2-2_CONV_1',
 'BLOCK_2-2_BN_1',
 'BLOCK_2-2_ACT_1',
 'BLOCK_2-2_ACT_QUANT',
 'BLOCK_2-2_CONV_3',
 'BLOCK_2-2_BN_3',
 'BLOCK_2-2_ADD',
 'BLOCK_2-3_CONV_1',
 'BLOCK_2-3_BN_1',
 'BLOCK_2-3_ACT_1',
 'BLOCK_2-3_ACT_QUANT',
 'BLOCK_2-3_CONV_3',
 'BLOCK_2-3_BN_3',
 'BLOCK_2-3_ADD',
 'BLOCK_2-4_CO

In [24]:
pb_model.model.layers[1].get_config()

{'name': 'ADVANCED_SPEC1',
 'trainable': True,
 'dtype': 'float32',
 'data_format': 'channels_last',
 'sample_rate': 48000,
 'spec_shape': [64, 512],
 'frame_step': 280,
 'fmin': 150,
 'fmax': 12000,
 'frame_length': 512}

In [34]:
pb_model.model.layers[1].input

<KerasTensor: shape=(None, 144000) dtype=float32 (created by layer 'INPUT')>

In [60]:
def printTensors(pb_file):
    # read pb into graph_def

    with tf.io.gfile.GFile(pb_file, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    # import graph_def
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)

    # print operations
    for op in graph.get_operations():
        print(op.name)

In [61]:
path_pb = "/home/emm06/birdclef-2023/vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"

Trying Onnyx...

In [35]:
!python -m tf2onnx.convert --saved-model ../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/ --output data/logs/BirdNet0 --opset 17

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-02-20 04:52:54,801 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2023-02-20 04:52:55,124 - WARNING - Importing a function (__inference_BLOCK_4-2_ACT_1_layer_call_and_return_conditional_losses_15267) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2023-02-20 04:52:55,164 - WARNING - Importing a function (__inference_BLOCK_4-3_ACT_1_layer_call_and_return_conditional_losses_33161) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2023-02-20 04:52:55,180 - WARNING - Importing a function (__inference_BLOCK_3-3_ACT_2_layer_call_and_return_conditional_losses_14525) with ops with unsaved custom gradients. Will likely fail if a gradient is requested.
2023-

In [43]:
!python -m tf2onnx.convert --opset 17 --tflite ../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model_FP32.tflite --output model.onnx

/opt/conda/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2023-02-20 04:55:57,760 - INFO - Using tensorflow=2.11.0, onnx=1.13.0, tf2onnx=1.13.0/2c1db5
2023-02-20 04:55:57,761 - INFO - Using opset <onnx, 17>
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2023-02-20 04:55:58,125 - ERROR - Failed to convert node 'model/ADVANCED_SPEC1/stft/rfft3' (fct=<bound method RFFT2DOp.version_13 of <class 'tf2onnx.onnx_opset.signal.RFFT2DOp'>>)
'OP=RFFT2D\nName=model/ADVANCED_SPEC1/stft/rfft3\nInputs:\n\tmodel/ADVANCED_SPEC1/stft/rfft2=Unsqueeze, [-1, -1, 1, 512], 1\n\tmodel/ADVANCED_SPEC1/stft/rfft1=Const, [2], 6\nOutpus:\n\tmodel/ADVANCED_SPEC1/stft/rfft3=[-1, -1, 1, 257], 14'
Traceback (most recent call last):
  File "/home/emm06/birdclef-2023/venv/lib/python3.7/site-packages/tf2onnx/tfonnx.py", line 292,